## Assignment: 
- Find a way to make topics more clear cut. (Cleaning, Stemming words, Removing Redundancy, ...)


In [70]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,HashingVectorizer
from sklearn.decomposition import LatentDirichletAllocation


In [71]:
tweet_df_politics_1 = pd.read_csv('hashtag_joebiden.csv').dropna()
tweet_df_politics_2 = pd.read_csv('hashtag_donaldtrump.csv', engine='python',error_bad_lines=False).dropna()
tweet_df_sport = pd.read_csv('tokyo_2020_tweets.csv').dropna()
tweet_df_general = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = 'latin-1',engine='python',error_bad_lines=False, header = None, names = ['target', 'ids', 'date', 'flag', 'user', 'text'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 15950: unexpected end of data
Skipping line 91677: unexpected end of data


In [72]:
sub_tweet_df_1 = tweet_df_politics_1.sample(n = 2000)
sub_tweet_df_1['text'] = sub_tweet_df_1['tweet']
sub_tweet_df_2 = tweet_df_politics_2.sample(n = 2000)
sub_tweet_df_2['text'] = sub_tweet_df_2['tweet']
sub_tweet_df_3 = tweet_df_general.sample(n = 10000)
sub_tweet_df_4 = tweet_df_sport.sample(n = 5000)

In [73]:
all_sub_tweets_df = pd.concat([sub_tweet_df_1['text'], sub_tweet_df_2['text'], sub_tweet_df_3['text'], sub_tweet_df_4['text']]).to_frame(name = 'text')
all_sub_tweets_df.head()

,text
42838,@Workplacesols I wonder has he ever looked up ...
14591,https://t.co/s7nv86OJJP\n\nI got a question to...
25612,My paintings of #DonaldTrump and #JoeBiden are...
49193,यह प्रयोग खासकर वोट बैंक के लिहाज से महत्वपूर्...
48144,#BidenHarris \n#BidenHarrisLandslide2020 \n#Bi...


In [74]:
all_sub_tweets_df[15000:]

,text
14357,I'm sure #India hockey 🏒 team will come back w...
2786,Im not ‘The Second Lasha Talakhadze.’ \nI am ‘...
6493,#Tokyo2020 Weightlifter Mirabai Chanu Wins Sil...
3394,BLACKSTENIUS... Again ! #Tokyo2020 👏👏
15082,Phew. Time to pop kettle on. If #fra v #usa i...
...,...
10163,Heartly Congratulations to @mirabai_chanu 🤗❤\n...
2824,3x3 basketball is where it's at #Tokyo2020
1015,🌷🌷🌷🌷🌷🌷🌷🌷🌷🌷\n\nMany congratulations to #Mirabai...
403,6-6. Third set tie-break to decide this bonker...


In [75]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

all_sub_tweets_df['text_lemmatized'] = all_sub_tweets_df.text.apply(lemmatize_text)
#df = all_sub_tweets_df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [76]:
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words = 'english', max_df = 0.7)
tfidf_tokens = tfidf_vect.fit_transform(all_sub_tweets_df['text'])
# tfidf_tokens = tfidf_vect.fit_transform(all_sub_tweets_df['text_tokenized']) #if you have some custom cleaning function

tfidf_vocab = tfidf_vect.get_feature_names_out()

In [77]:
lda = LatentDirichletAllocation(n_components = 10, doc_topic_prior = 5, topic_word_prior= 0.01)
lda.fit(tfidf_tokens)

LatentDirichletAllocation(doc_topic_prior=5, topic_word_prior=0.01)

In [78]:
topic_words = {}
n_top_words = 15
for topic, comp in enumerate(lda.components_):
    word_idx = np.argsort(comp)[::-2][:n_top_words]
    # store the words most relevant to the topic
    topic_words[topic] = [tfidf_vocab[i] for i in word_idx]
    
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))

Topic: 0
  tokyo2020, trump, like, work, today, time, don, bed, im, think, sleep, getting, going, got, gop
Topic: 1
  https, trump, day, olympics, just, going, really, medal, joebiden, sad, mirabaichanu, like, gone, winning, time
Topic: 2
  tokyo2020, olympics, medal, just, day, biden, time, proud, good, going, mirabai_chanu, know, mirabai, love, right
Topic: 3
  tokyo2020, trump, biden, work, good, congratulations, know, india, joebiden, like, home, mirabaichanu, school, time, thematildas
Topic: 4
  https, olympics, biden, india, work, want, day, joebiden, tomorrow, amp, mirabai_chanu, bad, won, need, sad
Topic: 5
  https, work, olympics, today, trump, mirabaichanu, india, amp, medal, miss, day, time, tokyoolympics, good, got
Topic: 6
  tokyo2020, trump, day, silver, olympics, joebiden, today, going, mirabai_chanu, home, proud, need, http, got, think
Topic: 7
  tokyo2020, trump, medal, just, joebiden, india, congratulations, today, good, like, miss, proud, football, team, women
Topic:


# Assignment: Reddit API

- Define your own topics and select a few popular threads from those topics on Reddit
- Retrieve the comment data using Praw API
- Group comments together as one corpus
- Perform Topic Modeling using LDA with various `n` topics.


In [79]:
# !pip install praw
!pip install asyncpraw 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [80]:
# import praw
import asyncpraw as praw
from asyncpraw.models import MoreComments
reddit = praw.Reddit(
    user_agent="Comment Extraction (by u/USERNAME)",
    client_id="eUqlLaI6IN7JksC5J_W7bg",
    client_secret="osv7c3tMHW9BLuVk8cJFB5i51c3YAg",
    redirect_uri='http://127.0.0.1:65011/'
                                 'authorize_callback',
     username="Srinivas_KDM",
     password="Rahul@0457"
)

In [81]:
import time
from tqdm.notebook import tqdm
topic_library = {
    'UMKC' : [ 'https://www.reddit.com/r/CollegeBasketball/comments/wde7py/cartiers_daily_throwback_highlight_day_8_chicago/', 
                  'https://www.reddit.com/r/ea2kcbb/comments/x1ubt9/year_6_at_umkc_and_we_are_ranked_for_the_first/',
                  'https://www.reddit.com/r/predental/comments/xk8gl8/does_umkc_accept_ap_credit_on_transcripts/'],
    'UIUC': [
        'https://www.reddit.com/r/ApplyingToCollege/comments/x95dlk/i_viewed_my_uiuc_csstat_admission_file/',
        'https://www.reddit.com/r/HostileArchitecture/comments/xbuwxu/some_of_the_only_benches_available_around_uiuc_an/',
        'https://www.reddit.com/r/chanceme/comments/xlmq3s/chance_asian_male_at_hypsm_cornell_vanderbilt/',
    ],
    'UCSB': [
        'https://www.reddit.com/r/UCSantaBarbara/comments/xbc6ll/isla_vista_and_ucsb_are_the_best_places_on_earth/',
        'https://www.reddit.com/r/UCSantaBarbara/comments/x9ng2b/do_students_actually_wear_ucsb_sweatshirts_on/', 
        'https://www.reddit.com/r/UCSantaBarbara/comments/xhw6pb/what_does_the_typical_ucsb_student_carry_with/',
    ]
}
for topic, urls in tqdm(topic_library.items()):
  for url in tqdm(urls):
    submission = await reddit.submission(url=url)
    submission.comments.replace_more(limit=0) # flatten tree
    all_comments = []
    comments = submission.comments # all comments
    # comments = submission.comments.list() # all comments
    for top_level_comment in submission.comments:
      if isinstance(top_level_comment, MoreComments):
          continue
      all_comments.append(top_level_comment.body)
      with open('{}-reddit_author.txt'.format(topic), 'a') as f:
          f.write(submission.selftext + '\n')
      with open('{}-reddit_comments.txt'.format(topic), 'a') as f:
        for i, comment in enumerate(comments):
          f.write(comment.body + '\n')
    # time.sleep(1.5)
    
  

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: coroutine 'CommentForest.replace_more' was never awaited


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [82]:
reddit_df = pd.DataFrame(columns = ['text'])
for topic, _ in tqdm(topic_library.items()):
  with open('{}-reddit_comments.txt'.format(topic)) as f:
    lines = f.readlines()
    reddit_sub_df = pd.DataFrame(lines, columns = ['text'])
    reddit_sub_df = reddit_sub_df.sample(3000, replace=True)
    reddit_df= reddit_df.append(reddit_sub_df).reset_index(drop=True)
# lines

  0%|          | 0/3 [00:00<?, ?it/s]

In [83]:
reddit_df

,text
0,Amazing turnaround!!\n
1,Why were there so many people there? Is chicag...
2,I'm more shocked that there are so many people...
3,https://www.youtube.com/watch?v=fkWOxWX4GP0\n
4,Incredible... Both shots!\n
...,...
8995,"One time, I wore the full outfit and then a gr..."
8996,My backpack had:\n
8997,Beyond the laptop you said you’d bring to take...
8998,* TI Nspire CX graphing calculator\n


In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer='word', stop_words = 'english', max_df = 0.7)
tfidf_tokens = tfidf_vect.fit_transform(reddit_df['text'])

tfidf_vocab = tfidf_vect.get_feature_names_out()

In [85]:
print(tfidf_tokens[0])
tfidf_vocab[100]

  (0, 1079)	0.7071067811865476
  (0, 65)	0.7071067811865476


'awareness'

In [86]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 10, doc_topic_prior=1)
lda.fit(tfidf_tokens)

import numpy as np 
topic_words = {}
n_top_words = 15
for topic, comp in enumerate(lda.components_):
    word_idx = np.argsort(comp)[::-1][:n_top_words]
    # store the words most relevant to the topic
    topic_words[topic] = [tfidf_vocab[i] for i in word_idx]
    
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))

Topic: 0
  did, comic, sans, thank, wow, useful, really, baselines, incredible, chicago, ot, drew, kevin, play, stands
Topic: 1
  state, 2005, wear, looks, years, quality, chicago, video, think, earlier, 10, oregon, usc, poor, class
Topic: 2
  college, chicago, state, isn, watch, wear, stop, white, players, black, cares, shocked, doesn, attendance, books
Topic: 3
  just, graduated, god, nooo, dear, state, chicago, umkc, really, don, type, need, school, additional, game
Topic: 4
  im, ao, year, people, sophomore, going, won, edit, state, engineering, videos, huh, game, doctor, moon
Topic: 5
  team, yes, algebra, sort, year, bad, basketball, game, real, love, promise, noise, tournament, showed, school
Topic: 6
  file, cs, insane, essays, gordon, bombay, right, hear, 31, holy, shit, good, view, coach, mean
Topic: 7
  merch, ucsb, students, 95, friend, 1570, rejected, got, uiuc, school, wear, seen, fun, water, study
Topic: 8
  turnaround, amazing, com, https, www, youtube, fkwoxwx4gp0, tim

In [87]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 25, doc_topic_prior=1)
lda.fit(tfidf_tokens)

import numpy as np 
topic_words = {}
n_top_words = 15
for topic, comp in enumerate(lda.components_):
    word_idx = np.argsort(comp)[::-1][:n_top_words]
    # store the words most relevant to the topic
    topic_words[topic] = [tfidf_vocab[i] for i in word_idx]
    
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))

Topic: 0
  sat, good, merch, tbh, fun, bro, feel, ur, afraid, heckling, scared, breakdown, score, want, watch
Topic: 1
  day, tourney, lol, poor, like, roos, hi, make, ll, conference, file, gets, apply, hopes, semis
Topic: 2
  personally, math, books, stats, media, whiteland, played, local, bygone, basketball, incredibly, david, just, respect, sense
Topic: 3
  charger, think, hs, support, yes, jerseys, jeez, white, useful, state, fixated, recalculate, looking, gpa, chicago
Topic: 4
  wear, im, bet, mit, cool, fucked, try, said, bad, like, imma, 40, shits, berkeley, file
Topic: 5
  stuff, sophomore, sure, ve, people, fkwoxwx4gp0, super, tryna, section, semester, talented, 1500, omg, party, kids
Topic: 6
  insightful, aos, sweatshirt, decision, admitted, welcome, job, supposed, girl, high, grainger, excellent, did, doctor, wear
Topic: 7
  huh, state, chicago, know, won, actually, stuff, sweater, clown, school, gum, case, seeing, interesting, engineering
Topic: 8
  black, backpack, chicag